
# Unified Clifford Computations — Notebook Scaffold

This notebook bundles minimal, **extensible** scaffolds for the computational pieces referenced in your paper draft.

**Contents**
1. Cauchy–Riemann residuals on slices  
2. Hyperbolic slice derivative test in $\operatorname{Cl}(1,1)$  
3. Cauchy projector (discrete) approximation on a slice  
4. Micro-benchmarks / complexity table

> **Note:** These are deliberately lightweight. Swap in your exact operators and kernels from your `CliffordLab` code where appropriate.



## Environment

- Python ≥ 3.9  
- Packages: `numpy`, `matplotlib`, `clifford`, `nbformat` (already present while generating this file).  
- If `clifford` is not installed, run the next cell.


In [ ]:

# If needed, uncomment to install. (May require internet & a proper env.)
# %pip install clifford numpy matplotlib --quiet


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Utilities
def show_image_grid(Z, xs, ys, title, cmap='viridis'):
    plt.figure(figsize=(6,5))
    plt.imshow(Z.T, extent=[xs[0], xs[-1], ys[0], ys[-1]], origin='lower', cmap=cmap, aspect='auto')
    plt.colorbar(label='value')
    plt.title(title)
    plt.xlabel('x'); plt.ylabel('y')
    plt.show()



---
## 1) Cauchy–Riemann residuals on slices

A thin numerical check of a CR-like residual using forward differences along a chosen slice direction.  
This is **illustrative** — replace the finite-difference and residual with your canonical definitions from the paper / library.


In [ ]:

from clifford import Cl

def make_clifford(p, q):
    layout, blades = Cl(p, q)
    return layout, blades

# Example: polynomial f(z) = z^2 + a z + b (acts by GA multiplication)
def poly_eval(z, a=1.0, b=0.0):
    return z*z + a*z + b

def cr_residuals(layout, blades, f, grid=(-1.5,1.5,-1.5,1.5,150), slice_key='e1', diff_key='e2'):
    e1 = blades[slice_key]
    e2 = blades[diff_key]
    xs = np.linspace(grid[0], grid[1], grid[4])
    ys = np.linspace(grid[2], grid[3], grid[4])
    R = np.zeros((len(xs), len(ys)))
    h = 1e-4
    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            z = x + y*e1
            # directional derivative along e2 (toy proxy for a CR component)
            df = (f(z + h*e2) - f(z)) / h
            # norm() exists on multivectors in 'clifford'
            R[i, j] = float(df.norm())
    return xs, ys, R

# Run on Cl(2,0) with a quadratic
layout, blades = make_clifford(2, 0)
xs, ys, R = cr_residuals(layout, blades, lambda z: poly_eval(z, a=1.0, b=1.0))
show_image_grid(R, xs, ys, "CR-like residual (proxy) for quadratic on Cl(2,0)")



---
## 2) Hyperbolic slice derivative test in $\mathrm{Cl}(1,1)$

Checks that the forward-difference derivative of the slice-exponential along a hyperbolic direction matches the expected series behavior.


In [ ]:

layout11, blades11 = make_clifford(1, 1)
e1, e2 = blades11['e1'], blades11['e2']

def exp_series(x, terms=20):
    out = 0
    for k in range(terms):
        out = out + (x**k)/np.math.factorial(k)
    return out

x_vals = np.linspace(-1.5, 1.5, 80)
h = 1e-4
num_deriv = []
series_eval = []

for x in x_vals:
    z = x*e1           # move along hyperbolic slice
    fz = exp_series(z, terms=30)
    df_num = (exp_series(z + h*e1, terms=30) - fz) / h  # directional derivative
    num_deriv.append(float(df_num[()]))   # scalar projection for a quick visual
    series_eval.append(float(fz[()]))

plt.figure(figsize=(6,4))
plt.plot(x_vals, num_deriv, label='numeric derivative (along e1)', linewidth=2)
plt.plot(x_vals, series_eval, '--', label='series value', linewidth=2)
plt.title("Hyperbolic slice derivative test (Cl(1,1))")
plt.xlabel('x'); plt.ylabel('value (scalar projection)'); plt.legend(); plt.show()



---
## 3) Cauchy projector (discrete) approximation on a slice

A classic complex-plane Cauchy reconstruction (circle contour + trapezoid).  
In your framework: run per-slice (elliptic/hyperbolic) with the **appropriate kernels and contours**.


In [ ]:

def cauchy_projector_complex(f, pts, contour_pts=200, R=2.0):
    theta = np.linspace(0, 2*np.pi, contour_pts, endpoint=False)
    circle = R*np.exp(1j*theta)                # parameterized contour in C
    dz = 1j*R*np.exp(1j*theta) * (2*np.pi/contour_pts)  # dz for trapezoid
    f_vals = f(circle)
    results = []
    for z in pts:
        integrand = f_vals / (circle - z)
        val = np.sum(integrand * dz) / (2j*np.pi)
        results.append(val)
    return np.array(results)

# Test: recover polynomial inside contour
f = lambda z: z**2 + 1 + 0.2*z
pts = np.linspace(-0.9, 0.9, 200)
proj_vals = cauchy_projector_complex(f, pts, contour_pts=1024, R=1.5)

plt.figure(figsize=(6,4))
plt.plot(pts, f(pts).real, label="original (real)")
plt.plot(pts, proj_vals.real, '--', label="reconstructed (real)")
plt.title("Cauchy projector on circle (complex slice demo)")
plt.xlabel('x'); plt.ylabel('Re f'); plt.legend(); plt.show()



---
## 4) Micro-benchmarks / complexity table

Crude timings for polynomial evaluation and a toy projector. Replace with kernels from your package to get realistic profiles.


In [ ]:

import time

def timing_poly_eval(degrees=(2, 4, 8, 16, 32), repeats=2000):
    rows = []
    z = 0.7 + 0.2j
    for d in degrees:
        f = lambda w: sum(w**k for k in range(d+1))
        t0 = time.time()
        for _ in range(repeats):
            _ = f(z)
        t1 = time.time()
        rows.append((d, (t1 - t0)/repeats))
    return rows

rows = timing_poly_eval()
print("Degree | Avg time per eval (s)")
for d, t in rows:
    print(f"{d:>6} | {t:.3e}")



---

### Notes & Next Steps

- The CR residual in §1 is a **proxy**. Replace with your unified CR residuals for $J^2=\pm 1$ and visualize side-by-side.
- In §2, you can compare against your closed forms (e.g., $e^x \cosh y$, $e^x \sinh y$) on the hyperbolic slice.
- In §3, swap the complex kernel with your **slice-wise Cauchy kernels** (elliptic circle vs. hyperbolic rectangle contours).
- In §4, extend timings to cover: (i) series truncation error vs. cost, (ii) projector quadrature points vs. error/cost.

Happy hacking! :)
